In [4]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP



In [ ]:
"""def attention(Q, K, V):
    dimension = Q.shape[1]
    attention=np.dot(Q, K.T)
    is_all =attention/np.sqrt(dimension)
    you = softmax(is_all)
    need = np.dot(you, V)
    return need"""

In [ ]:
"""def attention(Q,K,V):
    m, n = Q.size()
    attention = torch.matmul(Q, K.T)
    is_all = attention/torch.sqrt(torch.tensor(n))
    you = torch.nn.functional.softmax(is_all, dim=-1)
    need = torch.matmul(torch.tensor(you), V)
    return need"""


In [ ]:
Q = torch.randn(4,3)
K = torch.randn(4,3)
V = torch.randn(4,3)

In [ ]:
"""def feed_forward(Q,K,V):
    m, n = Q.size()
    attention = torch.matmul(Q, K.T)
    is_all = attention/torch.sqrt(torch.tensor(n))
    you = torch.nn.functional.softmax(is_all, dim=0)
    need = torch.matmul(torch.tensor(you), V)
    linear1 = torch.nn.Linear(n, 2048)(need)
    relu = torch.nn.ReLU()(linear1)
    linear2 = torch.nn.Linear(2048, 512)(relu)
    return linear2
"""

In [ ]:
#feed_forward(Q,K,V)

/var/folders/wy/zrk8x2415z7cybpg4y_2chvm0000gn/T/ipykernel_37720/1859686510.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  need = torch.matmul(torch.tensor(you), V)


tensor([[-0.0512,  0.1466,  0.0654,  ...,  0.2638,  0.0680,  0.1026],
        [-0.0375,  0.0568, -0.0147,  ...,  0.2257, -0.0017,  0.0132],
        [ 0.0229,  0.0365,  0.0768,  ...,  0.4782, -0.0735,  0.0136],
        [-0.1525,  0.1812, -0.0339,  ...,  0.0854,  0.2075,  0.1902]],
       grad_fn=<AddmmBackward0>)

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value):
        "Implements Figure 2"
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        """        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]"""
        #.view(nbatches, -1, self.h, self.d_k).transpose(1,2)
        QKVList = [query, key, value]
        for i in range(len(lin)):
          for lin, x in zip(self.linears, QKVList[i]):
            QKVList[i] = [lin(x)]
        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(
            query, key, value, mask=None, dropout=self.dropout
        )
        # 3) "Concat" using a view and apply a final linear.
        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        del query
        del key
        del value
        return self.linears[-1](x)

In [5]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])